In [ ]:
# 2022A 4)结果

In [115]:
"""
问题	入射波浪频率 (s-1)	垂荡附加质量 (kg)	纵摇附加转动惯量 (kg·m2)	垂荡兴波阻尼系数 (N·s/m)	纵摇兴波阻尼系数 (N·m·s)	垂荡激励力振幅 (N)	纵摇激励力矩振幅 (N·m)
问题1	1.4005	1335.535	6779.315	656.3616	151.4388	6250	1230
问题2	2.2143	1165.992	7131.29	    167.8395	2992.724	4890	2560
问题3	1.7152	1028.876	7001.914	683.4558	654.3383	3640	1690
问题4	1.9806	1091.099	7142.493	528.5018	1655.909	1760	2140
"""
pass

In [116]:
"""
参数	取值
浮子质量 (kg)	4866
浮子底半径 (m)	1
浮子圆柱部分高度 (m)	3
浮子圆锥部分高度 (m)	0.8
振子质量 (kg)	2433
振子半径 (m)	0.5
振子高度 (m)	0.5
海水的密度 (kg/m3)	1025
重力加速度 (m/s2)	9.8
弹簧刚度 (N/m)	80000
弹簧原长 (m)	0.5
扭转弹簧刚度 (N·m)	250000
静水恢复力矩系数 (N·m)	8890.7
"""
pass

In [117]:
import numpy as np

In [118]:
rho = 1025
g = 9.8
rho_g_pi_r2 = rho * g * np.pi * (1)**2

In [119]:
I_floater = 8289.43
I_floater_extra = 7142.493
I_oscillator = lambda x: 202.75 + 2443 * (0.75 + x)

m_oscillator = 2433
m_floater = 4866
m_floater_extra = 1091.099

k = 80000
g = 250000
wave_g = 8890.7
wave_fre = 1.9806
wave_force_max = 1760
wave_torque_max = 2140

gamma_wave = 528.5018

epsilon_wave = 1655.909

In [120]:
time = 40 * 2 * np.pi / wave_fre
dt = 0.05
n_ = int(time / dt) + 1

In [121]:
time

126.89458360455592

In [122]:
def output(gamma, epsilon):
    ksi = np.zeros(n_)
    x = np.zeros(n_)
    theta = np.zeros(n_)
    t = np.zeros(n_)
    ret = 0
    for t_ in range(2, n_):
        spring_force = -k * (x[t_ - 1] - ksi[t_ - 1])
        relative_speed = ((x[t_ - 1] - ksi[t_ - 1]) - (x[t_ - 2] - ksi[t_ - 2])) / dt
        damping_force = -gamma * np.sqrt(np.abs(relative_speed)) * relative_speed
        x[t_] = 2 * x[t_ - 1] - x[t_ - 2] + (spring_force + damping_force) * dt ** 2 / m_oscillator
        floating_force = -rho_g_pi_r2 * ksi[t_ - 1]
        wave_force = wave_force_max * np.cos(wave_fre * t_ * dt)
        secondary_wave_force = -gamma_wave * (ksi[t_ - 1] - ksi[t_ - 2]) / dt
        ksi[t_] = 2 * ksi[t_ - 1] - ksi[t_ - 2] + (secondary_wave_force + floating_force + wave_force - spring_force - damping_force) * dt ** 2 / (m_floater + m_floater_extra)
        ret += np.abs(damping_force * relative_speed * dt)
        
        ksi_acc = (ksi[t_] - 2 * ksi[t_ - 1] + ksi[t_ - 2]) / dt ** 2
        inertial_force = -m_floater_extra * ksi_acc
        inertial_torque = inertial_force * (x[t_] + 0.75 - 00.3) * np.sin(t[t_ - 1])
        
        spring_torque = -g * (t[t_ - 1] - theta[t_ - 1])
        relative_angular_speed = ((t[t_ - 1] - theta[t_ - 1]) - (t[t_ - 2] - theta[t_ - 2])) / dt
        damping_torque = -epsilon * relative_angular_speed
        t[t_] = 2 * t[t_ - 1] - t[t_ - 2] + (inertial_torque + spring_torque + damping_torque) * dt ** 2 / I_floater
        floating_torque = -wave_g * theta[t_ - 1]
        secondary_wave_torque = -epsilon_wave* (theta[t_ - 1] - theta[t_ - 2]) / dt
        wave_torque = wave_torque_max * np.cos(wave_fre * t_ * dt)
        theta[t_] = 2 * theta[t_ - 1] - theta[t_ - 2] + (secondary_wave_torque + floating_torque + wave_torque- spring_torque - damping_torque) * dt ** 2 / (I_floater + I_floater_extra)
        ret += np.abs(damping_torque * relative_angular_speed * dt)
    return ret

In [123]:
# ans_gamma = 68222.55406425311
ans_gamma = 100000
# ans_epsilon = 48124.56529121844
ans_epsilon = 30000
out = output(ans_gamma, ans_epsilon)
out

33933.02516902397

In [126]:
# simulated annealing
sim_T = 1
sim_step = 0.0001
iterations  = int((sim_T - sim_step) / sim_step)
sigma = 0.1
for i in range(iterations):
    next_gamma = ans_gamma + np.random.normal(0, sigma)
    next_gamma = np.clip(next_gamma, 0, 100000)
    next_epsilon = ans_epsilon + np.random.normal(0, sigma)
    next_epsilon = np.clip(next_epsilon, 0, 100000)
    next_out = output(next_gamma, next_epsilon)
    if next_out > out or np.random.rand() < np.exp((next_out - out) / sim_T):
        ans_gamma = next_gamma
        ans_epsilon = next_epsilon
        out = next_out
    sim_T -= sim_step
    if i % int(iterations / 10) == 0:
        print(f"""
        {i / iterations * 100}%
        gamma: {ans_gamma}
        epsilon: {ans_epsilon}
        out: {out} 
            """)


        0.0%
        gamma: 100000.0
        epsilon: 30419.09770485023
        out: 33934.11490363202 
            

        9.99099909990999%
        gamma: 99999.25758856852
        epsilon: 30413.69728730554
        out: 33934.02085775289 
            

        19.98199819981998%
        gamma: 99999.78264140386
        epsilon: 30417.862994164687
        out: 33934.088267615894 
            

        29.972997299729975%
        gamma: 99999.46328652282
        epsilon: 30418.286495119024
        out: 33934.05493252373 
            

        39.96399639963996%
        gamma: 99999.10769134483
        epsilon: 30420.5854173195
        out: 33934.0225509635 
            

        49.95499549954995%
        gamma: 99998.62960182858
        epsilon: 30420.74397392878
        out: 33933.971413446576 
            

        59.94599459945995%
        gamma: 99997.147898413
        epsilon: 30416.92096904957
        out: 33933.801740306175 
            

        69.93699369936994%
       